In [1]:
!git clone https://github.com/facebookresearch/PoincareMaps.git

Cloning into 'PoincareMaps'...


In [3]:
%pip install scanpy fastdtw stochman torchplot

^C
Note: you may need to restart the kernel to use updated packages.


In [4]:
%cd PoincareMaps

c:\Users\Prarabdha\Downloads\rvae-master\rvae-master\PoincareMaps


     ---------------------------------------- 2.0/2.0 MB 646.0 kB/s eta 0:00:00
     -------------------------------------- 133.4/133.4 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     -------------------------------------- 233.8/233.8 kB 1.2 MB/s eta 0:00:0

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Python310\\Lib\\site-packages\\scanpy\\_metadata.py'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
argilla 1.2.1 requires httpx<0.24,>=0.15, but you have httpx 0.24.1 which is incompatible.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 2.0.7 which is incompatible.
tensorflow 2.9.2 requires keras<2.10.0,>=2.9.0rc0, but you have keras 2.10.0 which is incompatible.
tensorflow 2.9.2 requires tensorboard<2.10,>=2.9, but you have tensorboard 2.10.0 which is incompatible.
tensorflow 2.9.2 requires tensorf

  Using cached scanpy-1.9.3-py3-none-any.whl (2.0 MB)
  Using cached fastdtw-0.3.4-py3-none-any.whl
  Using cached stochman-0.3.0-py3-none-any.whl
  Using cached torchplot-0.2.0-py3-none-any.whl
  Using cached statsmodels-0.14.0-cp310-cp310-win_amd64.whl (9.2 MB)
  Using cached session_info-1.0.0-py3-none-any.whl
  Using cached patsy-0.5.3-py2.py3-none-any.whl (233 kB)
  Using cached umap_learn-0.5.3-py3-none-any.whl
  Using cached anndata-0.9.2-py3-none-any.whl (104 kB)
  Using cached natsort-8.4.0-py3-none-any.whl (38 kB)
  Using cached numpy-1.23.5-cp310-cp310-win_amd64.whl (14.6 MB)
  Using cached numpy-1.22.4-cp310-cp310-win_amd64.whl (14.7 MB)
  Using cached pynndescent-0.5.10.tar.gz (1.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached stdlib_list-0.9.0-py3-none-any.whl (75 kB)
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55622 sha256=25fe024dde173a73aad7cc027d37117894ef

In [5]:
import scanpy as sc
import pandas as pd
import numpy as np

from main import *
from poincare_maps import *
# from PoincareMaps.main import *

%matplotlib inline
# from PoincareMaps.poincare_maps import *

In [8]:
dataset_fullpath = './datasets/Paul'
features, labels = prepare_data(dataset_fullpath,
                                with_labels=True,
                                normalize=True,
                                n_pca=0)

In [9]:
features,labels

(tensor([[-0.7070, -0.1914, -0.2720,  ..., -0.4816, -0.4488, -0.2217],
         [ 0.3422, -0.1914, -0.2720,  ...,  1.6369, -0.4488, -0.2217],
         [ 0.1302, -0.1914, -0.2720,  ...,  1.2589,  0.6859, -0.2217],
         ...,
         [-0.7070, -0.1914,  5.6759,  ..., -0.4816, -0.4488, -0.2217],
         [ 2.1656, -0.1914,  1.4426,  ..., -0.4816, -0.4488, -0.2217],
         [-0.0295, -0.1914, -0.2720,  ..., -0.4816, -0.4488,  1.3923]],
        dtype=torch.float64),
 array(['7MEP', '15Mo', '3Ery', ..., '7MEP', '15Mo', '3Ery'], dtype='<U7'))

In [10]:
%cd ..

c:\Users\Prarabdha\Downloads\rvae-master\rvae-master


In [11]:
features.shape

torch.Size([2730, 1000])

In [12]:
import os
import torch
import numpy as np

from itertools import chain

from rvae.geoml import nnj
from rvae.variational_inference.train import train_rvae, test_rvae, train_vae, test_vae
from rvae.utils.data_utils import get_mnist_loaders, get_fmnist_loaders, get_kmnist_loaders
from rvae.models.vae import RVAE, VAE
from rvae.utils.save_utils import save_model, load_model
from torch.utils.data import DataLoader, Dataset


class Experiment():
    def __init__(self,features):
        # self.dataset = "mnist"

        features = features.type(torch.float32)
        self.train_loader = DataLoader(features, batch_size=100, shuffle=True)
        self.test_loader = DataLoader(features, batch_size=100, shuffle=True)
        in_dim = 1000

        if not os.path.exists("./data/"):
            os.makedirs("./data/")
        # if self.dataset == "mnist":
        #     self.train_loader, self.test_loader = get_mnist_loaders("./data/", 100)
        #     in_dim = 784
        # elif self.dataset == "fmnist":
        #     self.train_loader, self.test_loader = get_fmnist_loaders("./data/", 100)
        #     in_dim = 784
        # elif self.dataset == "kmnist":
        #     self.train_loader, self.test_loader = get_kmnist_loaders("./data/", 100)
        #     in_dim = 784

        self.rvae_save_dir = os.path.join("./saved_models/", "RVAE/")
        self.vae_save_dir = os.path.join("./saved_models/", "VAE/")
        if not os.path.exists(self.rvae_save_dir):
            os.makedirs(self.rvae_save_dir)
        if not os.path.exists(self.vae_save_dir):
            os.makedirs(self.vae_save_dir)

        if not os.path.exists("./results/"):
            os.makedirs("./results/")
            # create graph directory
            os.makedirs("./results/" + "graphs/")
            # create model samples directory
            os.makedirs("./results/" + "samples/")

        # if args.seed is not None:
        #     assert(type(args.seed) == int)
        #     torch.manual_seed(args.seed)
        #     np.random.seed(args.seed)

        self.model = RVAE(in_dim, 2, 64, [512,256,128],
                              [128,256,512], nnj.Softplus, nnj.Sigmoid, 0.01, 1e-9)
        # if args.model.lower() == "rvae":
        #     self.model = RVAE(in_dim, args.latent_dim, args.num_centers, args.enc_layers,
        #                       args.dec_layers, nnj.Softplus, nnj.Sigmoid, args.rbf_beta, args.rec_b)
        # elif args.model.lower() == "vae":
        #     self.model = VAE(in_dim, args.latent_dim, args.num_centers, args.num_components,
        #                      args.enc_layers, args.dec_layers, nnj.Softplus, nnj.Sigmoid,
        #                      args.rbf_beta, args.rec_b)

        self.batch_size = 100
        self.mu_epochs = 100
        self.sigma_epochs = 100
        self.warmup_learning_rate = 1e-3
        self.sigma_learning_rate = 1e-3
        self.log_invl = 100
        self.save_invl = 25
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def train(self):
        self.model = self.model.to(self.device)
        # ================= RVAE =================
        if isinstance(self.model, RVAE):
            warmup_optimizer = torch.optim.Adam(
                chain(
                    self.model.encoder.parameters(),
                    self.model.q_mu.parameters(),
                    self.model.q_t.parameters(),
                    self.model.p_mu.parameters()),
                lr=self.warmup_learning_rate
            )
            sigma_optimizer = torch.optim.Adam(
                chain(
                    self.model.p_sigma.parameters(),
                    [self.model.pr_means, self.model.pr_t]),
                lr=self.sigma_learning_rate
            )

            # encoder/decoder mean optimization
            for epoch in range(1, self.mu_epochs + 1):
                loss, _, _ = train_rvae(epoch, self.train_loader, self.batch_size, self.model,
                                        warmup_optimizer, self.log_invl, self.device)
                print("\tEpoch: {} (warmup phase), negative ELBO: {:.3f}".format(epoch, loss))

            # warmup checkpoint
            savepath = os.path.join(self.rvae_save_dir, "Paul_warmup")
            save_model(self.model, sigma_optimizer, 0, None, savepath)

            self.model.switch = False
            self.model._update_latent_codes(self.train_loader)
            self.model._update_RBF_centers(beta=0.01)
            self.model._mean_warmup = False
            self.model._initialize_prior_means()

            # decoder sigma/prior parameters optimization
            for epoch in range(1, self.sigma_epochs + 1):
                loss, _, _ = train_rvae(epoch, self.train_loader, self.batch_size, self.model,
                                        sigma_optimizer, self.log_invl, self.device)
                print("\tEpoch: {} (sigma optimization), negative ELBO: {:.3f}".format(epoch, loss))

            savepath = os.path.join(self.rvae_save_dir,
                                    "Paul_epoch"+str(epoch)+"ckpt")
            save_model(self.model, sigma_optimizer, epoch, loss, savepath)

        # ================= VAE =================
        else:
            warmup_optimizer = torch.optim.Adam(
                chain(self.model.encoder.parameters(),
                      self.model.q_mu.parameters(),
                      self.model.q_var.parameters(),
                      self.model.decoder.parameters(),
                      self.model.p_mu.parameters()),
                      lr=self.warmup_learning_rate
            )

            if self.model.num_components > 1:
                sigma_optimizer = torch.optim.Adam(
                    chain(self.model.p_sigma.parameters(), self.model.means.parameters()),
                    lr=self.sigma_learning_rate
                )
            else:
                sigma_optimizer = torch.optim.Adam(
                    self.model.p_sigma.parameters(),
                    lr=self.sigma_learning_rate
                )

            # encoder/decoder mean optimization
            for epoch in range(1, self.mu_epochs + 1):
                loss, _, _ = train_vae(epoch, self.mu_epochs, self.train_loader, self.batch_size, self.model,
                                       warmup_optimizer, self.log_invl, self.device)
                print("\tEpoch: {} (warmup phase), negative ELBO: {:.3f}".format(epoch, loss))

            # warmup checkpoint
            savepath = os.path.join(self.rvae_save_dir, "Paul_warmup")
            save_model(self.model, sigma_optimizer, 0, None, savepath)

            self.model.switch = False
            self.model._update_latent_codes(self.train_loader)
            self.model._update_RBF_centers(beta=0.01)

            for epoch in range(1, self.sigma_epochs + 1):
                loss, _, _ = train_vae(epoch, 1, self.train_loader, self.batch_size, self.model,
                                       sigma_optimizer, self.log_invl, self.device)
                print("\tEpoch: {} (sigma optimization), negative ELBO: {:.3f}".format(epoch, loss))

                if epoch % self.save_invl == 0:
                    savepath = os.path.join(self.vae_save_dir,
                                            self.dataset+"_K"+str(self.model.num_components)+"epoch"+str(epoch)+".ckpt")
                    save_model(self.model, sigma_optimizer, epoch, loss, savepath)

    def eval(self, pretrained_path=None):
        # load checkpoint
        if pretrained_path is not None:
            placeholder_optimizer = torch.optim.Adam(
                self.model.p_sigma.parameters(),
                lr=1e-3
            )
            load_model(pretrained_path, self.model, placeholder_optimizer)

        if isinstance(self.model, RVAE):
            loss, log_cond, KL = test_rvae(self.test_loader, self.batch_size, self.model, self.device)
        else:
            loss, log_cond, KL = test_vae(self.test_loader, self.batch_size, self.model, self.device)

        print("Test set negative ELBO: {:.3f}, negative conditional: {:.3f}, KL: {:.3f}".format(loss, log_cond, KL))

In [13]:
experiment = Experiment(features)

In [14]:
experiment.train()

Epoch: 1, batch: 0, loss: 1522.985, KL: 0.000
Epoch: 1, batch: 0, loss: 1520.560, KL: 0.000
Epoch: 1, batch: 0, loss: 1432.424, KL: 0.000
Epoch: 1, batch: 0, loss: 1441.876, KL: 0.000
Epoch: 1, batch: 0, loss: 1453.704, KL: 0.000
Epoch: 1, batch: 0, loss: 1422.975, KL: 0.000
Epoch: 1, batch: 0, loss: 1412.837, KL: 0.000
Epoch: 1, batch: 0, loss: 1403.263, KL: 0.000
Epoch: 1, batch: 0, loss: 1430.549, KL: 0.000
Epoch: 1, batch: 0, loss: 1414.766, KL: 0.000
Epoch: 1, batch: 0, loss: 1388.730, KL: 0.000
Epoch: 1, batch: 0, loss: 1429.361, KL: 0.000
Epoch: 1, batch: 0, loss: 1404.777, KL: 0.000
Epoch: 1, batch: 0, loss: 1411.184, KL: 0.000
Epoch: 1, batch: 0, loss: 1393.420, KL: 0.000
Epoch: 1, batch: 0, loss: 1427.881, KL: 0.000
Epoch: 1, batch: 0, loss: 1411.333, KL: 0.000
Epoch: 1, batch: 0, loss: 1424.740, KL: 0.000
Epoch: 1, batch: 0, loss: 1444.121, KL: 0.000
Epoch: 1, batch: 0, loss: 1408.823, KL: 0.000
Epoch: 1, batch: 0, loss: 1499.888, KL: 0.000
Epoch: 1, batch: 0, loss: 1419.800

AttributeError: 'Experiment' object has no attribute 'dataset'